In [ ]:

import sys
import os
import math
import random
import time
import datetime
import re
import json
import csv
import shutil
import glob
import subprocess
import collections
import itertools
import functools
import operator
import inspect
import threading
import multiprocessing
import queue
import socket
import urllib
import requests
import hashlib
import base64
import pickle
import gzip
import bz2
import lzma
import zlib
import xml
import html
import email
import sqlite3
import cryptography
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn
import tensorflow as tf
import torch
from matplotlib import pyplot
from matplotlib.image import imread
from distutils.dir_util import copy_tree
from shutil import copy
from tensorflow.keras import layers, models
from matplotlib import pyplot as plt
import keras as keras
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import tensorflow as tf
import tensorflow.keras.metrics as tfm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("badasstechie/coffee-leaf-diseases")

print("Path to dataset files:", path)

100%|██████████| 196M/196M [00:02<00:00, 101MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/badasstechie/coffee-leaf-diseases/versions/3


In [ ]:
dataset_path = '/root/.cache/kagglehub/datasets/badasstechie/coffee-leaf-diseases/versions/3/coffee-leaf-diseases'
print("Files in dataset directory:", os.listdir(dataset_path))

Files in dataset directory: ['train', 'test']


In [ ]:
# List the files in the folder to ensure correct file path
files_in_dataset = os.listdir(dataset_path)
print("Files in dataset directory:", files_in_dataset)

# If the 'coffee-leaf-diseases' folder exists, let's check its contents
if 'coffee-leaf-diseases' in files_in_dataset:
    coffee_leaf_folder = os.path.join(dataset_path, 'coffee-leaf-diseases')
    files_in_coffee_leaf = os.listdir(coffee_leaf_folder)
    print("Files in 'coffee-leaf-diseases' folder:", files_in_coffee_leaf)


Files in dataset directory: ['train', 'test']


In [ ]:
# Check the contents of 'train' and 'test' subfolders
train_folder = os.path.join(dataset_path, 'train')
test_folder = os.path.join(dataset_path, 'test')

train_files = os.listdir(train_folder)
test_files = os.listdir(test_folder)

print("Files in 'train' folder:", train_files)
print("Files in 'test' folder:", test_files)

Files in 'train' folder: ['masks', 'images']
Files in 'test' folder: ['masks', 'images']


In [ ]:
# Path to the dataset
dataset_path = '/root/.cache/kagglehub/datasets/badasstechie/coffee-leaf-diseases/versions/3/coffee-leaf-diseases'

# Load the CSV files
train_csv_path = os.path.join(dataset_path, 'train_classes.csv')
test_csv_path = os.path.join(dataset_path, 'test_classes.csv')

if os.path.exists(train_csv_path) and os.path.exists(test_csv_path):
    train = pd.read_csv(train_csv_path)
    test = pd.read_csv(test_csv_path)
    print(train.head(), test.head())

train_images_path = os.path.join(dataset_path, 'train', 'images')
test_images_path = os.path.join(dataset_path, 'test', 'images')

train_images_files = os.listdir(train_images_path)
test_images_files = os.listdir(test_images_path)

print(f"Train images: {train_images_files[:5]}")
print(f"Test images: {test_images_files[:5]}")

#Data Category
classification_directories = [
    '/kaggle/working/train/imagesm/miner', '/kaggle/working/train/imagesm/nominer',
    '/kaggle/working/test/imagesm/miner', '/kaggle/working/test/imagesm/nominer',
    '/kaggle/working/train/imagesr/rust', '/kaggle/working/train/imagesr/norust',
    '/kaggle/working/test/imagesr/rust', '/kaggle/working/test/imagesr/norust',
    '/kaggle/working/train/imagesp/phoma', '/kaggle/working/train/imagesp/nophoma',
    '/kaggle/working/test/imagesp/phoma', '/kaggle/working/test/imagesp/nophoma'
]

for directory in classification_directories:
    os.makedirs(directory, exist_ok=True)

for image_file in train_images_files:
    image_id = image_file.split('.')[0]
    label = 'miner'
    destination = '/kaggle/working/train/imagesm/miner' if label == 'miner' else '/kaggle/working/train/imagesm/nominer'
    copy(os.path.join(train_images_path, image_file), destination)

for image_file in test_images_files:
    image_id = image_file.split('.')[0]
    label = 'miner'
    destination = '/kaggle/working/test/imagesm/miner' if label == 'miner' else '/kaggle/working/test/imagesm/nominer'
    copy(os.path.join(test_images_path, image_file), destination)


Train images: ['1396.jpg', '473.jpg', '283.jpg', '165.jpg', '1076.jpg']
Test images: ['1175.jpg', '1274.jpg', '64.jpg', '556.jpg', '1659.jpg']


In [ ]:
# Information about the training dataset
print("Training Dataset Info:")
print(train.info())

# Information about the test dataset
print("\nTest Dataset Info:")
print(test.info())

# Check for duplicates in both training and test datasets
train_duplicates = train.duplicated().sum()
test_duplicates = test.duplicated().sum()

print(f"\nNumber of duplicates in train dataset: {train_duplicates}")
print(f"Number of duplicates in test dataset: {test_duplicates}")

# Check for null values in both training and test datasets
train_nulls = train.isnull().sum()
test_nulls = test.isnull().sum()

print("\nNull values in train dataset:")
print(train_nulls)

print("\nNull values in test dataset:")
print(test_nulls)

# Remove duplicates from both datasets
train_clean = train.drop_duplicates()
test_clean = test.drop_duplicates()

# Remove rows with null values from both datasets
train_clean = train_clean.dropna()
test_clean = test_clean.dropna()

# Confirm the cleaning by checking the info again
print("\nCleaned Training Dataset Info:")
print(train_clean.info())

print("\nCleaned Test Dataset Info:")
print(test_clean.info())

Training Dataset Info:


NameError: name 'train' is not defined

In [ ]:
train_clean.head()

In [ ]:
traindatasetm = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/working/train/imagesm',
    labels='inferred',
    label_mode="binary",  # Binary classification (miner vs. nominer)
    class_names=None,
    color_mode="rgb",
    batch_size=16,
    image_size=(1024, 1024),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

testdatasetm = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/working/test/imagesm',
    labels='inferred',
    label_mode="binary",  # Binary classification (miner vs. nominer)
    class_names=None,
    color_mode="rgb",
    batch_size=16,
    image_size=(1024, 1024),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 1264 files belonging to 2 classes.
Found 400 files belonging to 2 classes.


In [ ]:
print("All ", test.count())
print("Miner ", test[test.miner > 0 ].count())
print("Rust ", test[test.rust > 0 ].count())
print("Phoma ", test[test.phoma > 0 ].count())
print("None ", test[(test.miner == 0) & (test.rust == 0) & (test.phoma == 0) ].count())

All  id       400
miner    400
rust     400
phoma    400
dtype: int64
Miner  id       128
miner    128
rust     128
phoma    128
dtype: int64
Rust  id       72
miner    72
rust     72
phoma    72
dtype: int64
Phoma  id       96
miner    96
rust     96
phoma    96
dtype: int64
None  id       116
miner    116
rust     116
phoma    116
dtype: int64


In [ ]:
def capitalize_columns(df):
    df.columns = [col.strip().title() for col in df.columns]
    return df

train_clean = capitalize_columns(train_clean)
test_clean = capitalize_columns(test_clean)
train_clean = train_clean.rename(columns={'Id': 'ID'})
test_clean = test_clean.rename(columns={'Id': 'ID'})

# The updated column names
print("\nUpdated Column Names in Train Dataset:")
print(train_clean.columns)

print("\nUpdated Column Names in Test Dataset:")
print(test_clean.columns)


Updated Column Names in Train Dataset:
Index(['ID', 'Miner', 'Rust', 'Phoma'], dtype='object')

Updated Column Names in Test Dataset:
Index(['ID', 'Miner', 'Rust', 'Phoma'], dtype='object')


In [ ]:
merged_df = pd.concat([train_clean, test_clean], ignore_index=True)

print("\nMerged Dataset Info:")
print(merged_df.info())

print("\nFirst few rows of the merged dataset:")
print(merged_df.head())


Merged Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1664 entries, 0 to 1663
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ID      1664 non-null   int64
 1   Miner   1664 non-null   int64
 2   Rust    1664 non-null   int64
 3   Phoma   1664 non-null   int64
dtypes: int64(4)
memory usage: 52.1 KB
None

First few rows of the merged dataset:
   ID  Miner  Rust  Phoma
0   0      0     0      0
1   1      0     0      0
2   2      0     0      0
3   3      0     0      0
4   4      0     0      0


In [ ]:
print(merged_df)

        ID  Miner  Rust  Phoma
0        0      0     0      0
1        1      0     0      0
2        2      0     0      0
3        3      0     0      0
4        4      0     0      0
...    ...    ...   ...    ...
1659  1659      0     0      0
1660  1660      0     1      0
1661  1661      0     1      0
1662  1662      0     1      0
1663  1663      0     1      0

[1664 rows x 4 columns]
